In [23]:
from bs4 import BeautifulSoup
import requests
import selenium 
from urllib.parse import urljoin
from selenium import webdriver


In [17]:
website_url = 'https://www.wikiart.org/'
website_genre_page = urljoin(website_url, 'en/paintings-by-genre')
res = requests.get(base_link)
genre_page_soup = BeautifulSoup(res.text, 'html.parser')

In [22]:
genres_container = genre_page_soup.find_all('ul', 'dictionaries-list')[0] 
genre_links = genres_container.find_all('a')

# picking one - test
extension = genre_links[0]['href']
genre_url = urljoin(website_url, extension)

https://www.wikiart.org/en/paintings-by-genre/abstract?select=featured


* Selenium required to scrape all contents this dynamic webpage

In [40]:
driver = webdriver.Chrome()
driver.get(genre_url)
print(genre_url)

https://www.wikiart.org/en/paintings-by-genre/abstract?select=featured


In [41]:
genre_soup = BeautifulSoup(driver.page_source, 'html.parser')

In [46]:
# TO DO: Scroll down 
# select 1 title- testing 
title_blocks = genre_soup.find_all('div', "title-block")
print(f'Number of art titles {len(title_blocks)}')
title_block = title_blocks[0]
art_ref =  title_block.find('a', 'artwork-name ng-binding')
art_ref

Number of art titles 60


<a class="artwork-name ng-binding" href="/en/hakuin-ekaku/bonji" target="_self">Bonji </a>

In [47]:
art_url = urljoin(website, art_ref['href'])
html = requests.get(art_url).text
soup = BeautifulSoup(html, 'html.parser') 

def get_art_details(soup):
    pass


soup.find_all('img')

[<img alt="Bonji - Hakuin Ekaku" itemprop="image" src="https://uploads7.wikiart.org/images/hakuin-ekaku/bonji.jpg!Large.jpg" title="Bonji - Hakuin Ekaku"/>,
 <img alt="Buy Handmade Oil Painting Reproductions" src="https://uploads.wikiart.org/Content/wiki/img/order@2x.png"/>,
 <img alt="Chrysanthemums - Hakuin Ekaku" class="image-rendering-fix lazy-load" img-source="'https://uploads7.wikiart.org/images/hakuin-ekaku/chrysanthemums.jpg!Large.jpg'" lazy-load="" src="https://uploads.wikiart.org/Content/wiki/img/lazy-load-placeholder.png" title="Chrysanthemums - Hakuin Ekaku"/>,
 <img alt="Daruma in Red - Hakuin Ekaku" class="image-rendering-fix lazy-load" img-source="'https://uploads5.wikiart.org/images/hakuin-ekaku/daruma-in-red.jpg'" lazy-load="" src="https://uploads.wikiart.org/Content/wiki/img/lazy-load-placeholder.png" title="Daruma in Red - Hakuin Ekaku"/>,
 <img alt="Death - Hakuin Ekaku" class="image-rendering-fix lazy-load" img-source="'https://uploads6.wikiart.org/images/hakuin-ek

In [152]:
art_article = soup.find_all('article')[0]

art_title = art_article.h3.text
art_article

<article ng-init="trackPageView('painting', '5772853bedc2cb3880006428')">
<h3>Bonji</h3>
<h5 itemprop="creator" itemscope="" itemtype="https://schema.org/Person">
<span itemprop="name">
<a href="/en/hakuin-ekaku" target="_self">Hakuin Ekaku</a>
</span>
<meta content="http://en.wikipedia.org/wiki/Hakuin_Ekaku" itemprop="sameAs">
</meta></h5>
<ul>
<li class="dictionary-values">
<s>Style:</s>
<span>
<a href="/en/paintings-by-style/zen" target="_self">Zen</a>
</span>
</li>
<li class="dictionary-values">
<s>Genre:</s>
<span>
<a href="/en/paintings-by-genre/abstract" target="_self"><span itemprop="genre">abstract</span></a>
</span>
</li>
<li class="order-reproduction">
<a class="hidden-blank" href="https://www.1st-art-gallery.com/" ng-click="gaClick('order', 'order-click-page');" target="_blank">
<div class="order-reproduction-img-wrapper">
<img alt="Buy Handmade Oil Painting Reproductions" src="https://uploads.wikiart.org/Content/wiki/img/order@2x.png"/>
</div>
<h6>
                        

In [171]:
art_style_soup.s.text

'Style:'

In [175]:
art_style_soup = art_article.find_all('li', 'dictionary-values')[0]
assert 'style' in art_style_soup.s.text.lower()

art_style = art_style_soup.find('a').text


'Zen'

In [144]:
image_soups = soup.find_all('img')
image_soups[0]


<img alt="Bonji - Hakuin Ekaku" itemprop="image" src="https://uploads7.wikiart.org/images/hakuin-ekaku/bonji.jpg!Large.jpg" title="Bonji - Hakuin Ekaku"/>

In [137]:
image_soups = soup.find_all('img')
for s in image_soups:
    try:
        s['itemprop']
        break
    except:
        break
        
        
        
        
        

In [61]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
items1 = soup.findAll("a", {"class": "artwork-name ng-binding"})
items2 = soup.findAll("a", {"class": "artist-name ng-binding"})
assert len(items1) == len(items2)
